In [1]:
import json
import hashlib
from datetime import datetime
from breeze_connect import BreezeConnect
import urllib
import pandas as pd
import numpy as np
import math
from trading_api.icici import icici
import datetime as dt

In [2]:
api_session='37289997'

In [3]:
icici1 = icici(api_session=api_session)
icici1.login()

Exception: Could not authenticate credentials. Please check token and keys

In [85]:
stock = 'ITC'
from_date = '2024-03-19'
to_date = '2024-03-20'
interval = '1minute'
exchange = 'NSE'

df = icici1.get_histroy_data_stock(stock,from_date,to_date,interval=interval,exchange=exchange)


couldnt perform on datetime
Error could not convert string to float: '2024-03-19 09:08:00'
couldnt perform on stock_code
Error could not convert string to float: 'ITC'
open complete
close complete
high complete
low complete


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    398 non-null    object 
 1   stock_code  398 non-null    object 
 2   open        398 non-null    float64
 3   close       398 non-null    float64
 4   high        398 non-null    float64
 5   low         398 non-null    float64
dtypes: float64(4), object(2)
memory usage: 18.8+ KB


In [87]:
df['date_time'] = pd.to_datetime(df['datetime'])

In [88]:
df

,datetime,stock_code,open,close,high,low,date_time
0,2024-03-19 09:08:00,ITC,418.00,418.00,418.00,418.00,2024-03-19 09:08:00
1,2024-03-19 09:15:00,ITC,418.15,418.40,418.40,418.00,2024-03-19 09:15:00
2,2024-03-19 09:16:00,ITC,418.40,418.15,418.95,418.00,2024-03-19 09:16:00
3,2024-03-19 09:17:00,ITC,418.35,418.65,418.75,418.10,2024-03-19 09:17:00
4,2024-03-19 09:18:00,ITC,418.65,419.00,419.30,418.45,2024-03-19 09:18:00
...,...,...,...,...,...,...,...
393,2024-03-19 15:56:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:56:00
394,2024-03-19 15:57:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:57:00
395,2024-03-19 15:58:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:58:00
396,2024-03-19 15:59:00,ITC,409.55,409.55,409.55,409.55,2024-03-19 15:59:00


In [89]:
df.drop(df.index[(df.date_time.dt.hour> 14) & (df.date_time.dt.minute >19)],inplace=True)

In [90]:
df['change'] = df['close']-df['open']

In [91]:
df['per_change'] = (df['change']/df['open'])*100

In [92]:
df['per_change'].mean()

-0.006618667089363048

In [94]:
df['change'].mean()

-0.02752043596730205

In [95]:
for index,row in df.iterrows():
    trade = pd.DataFrame()
    print(row['datetime'])

2024-03-19 09:08:00
2024-03-19 09:15:00
2024-03-19 09:16:00
2024-03-19 09:17:00
2024-03-19 09:18:00
2024-03-19 09:19:00
2024-03-19 09:20:00
2024-03-19 09:21:00
2024-03-19 09:22:00
2024-03-19 09:23:00
2024-03-19 09:24:00
2024-03-19 09:25:00
2024-03-19 09:26:00
2024-03-19 09:27:00
2024-03-19 09:28:00
2024-03-19 09:29:00
2024-03-19 09:30:00
2024-03-19 09:31:00
2024-03-19 09:32:00
2024-03-19 09:33:00
2024-03-19 09:34:00
2024-03-19 09:35:00
2024-03-19 09:36:00
2024-03-19 09:37:00
2024-03-19 09:38:00
2024-03-19 09:39:00
2024-03-19 09:40:00
2024-03-19 09:41:00
2024-03-19 09:42:00
2024-03-19 09:43:00
2024-03-19 09:44:00
2024-03-19 09:45:00
2024-03-19 09:46:00
2024-03-19 09:47:00
2024-03-19 09:48:00
2024-03-19 09:49:00
2024-03-19 09:50:00
2024-03-19 09:51:00
2024-03-19 09:52:00
2024-03-19 09:53:00
2024-03-19 09:54:00
2024-03-19 09:55:00
2024-03-19 09:56:00
2024-03-19 09:57:00
2024-03-19 09:58:00
2024-03-19 09:59:00
2024-03-19 10:00:00
2024-03-19 10:01:00
2024-03-19 10:02:00
2024-03-19 10:03:00


In [101]:
nd = pd.DataFrame(columns=df.columns)

In [103]:
nd = nd.append(df.sample)

AttributeError: 'DataFrame' object has no attribute 'append'

In [56]:
import yfinance as yf

# Ticker symbol for ITC
ticker_symbol = "^NSEBANK"  # For the National Stock Exchange of India

# Fetch historical data
data = yf.download(ticker_symbol, period="2d", interval="1m")

print(data)


[*********************100%%**********************]  1 of 1 completed

                                   Open          High           Low  \
Datetime                                                              
2024-03-19 09:15:00+05:30  46421.898438  46524.851562  46421.898438   
2024-03-19 09:16:00+05:30  46520.851562  46530.199219  46498.699219   
2024-03-19 09:17:00+05:30  46528.101562  46548.750000  46507.449219   
2024-03-19 09:18:00+05:30  46544.750000  46554.101562  46527.648438   
2024-03-19 09:19:00+05:30  46521.449219  46530.648438  46501.648438   
...                                 ...           ...           ...   
2024-03-20 11:58:00+05:30  46276.351562  46276.351562  46249.750000   
2024-03-20 11:59:00+05:30  46253.550781  46275.250000  46243.949219   
2024-03-20 12:00:00+05:30  46273.648438  46273.648438  46255.800781   
2024-03-20 12:01:00+05:30  46259.000000  46285.148438  46259.000000   
2024-03-20 12:02:00+05:30  46278.601562  46278.601562  46278.601562   

                                  Close     Adj Close  Volume  
Datetime    

In [57]:
data['Change']=data['Close']-data['Open']

In [58]:
data['Pre_Change']= (data['Change']/data['Open'])*100

In [59]:
old_price_long = 0
old_price_short = 0
target_prencantage = 0.1
stop_loss_precentage = 0.2 
long_hold = False
short_hold = False
trade_book = []
qty = 30
brokerage = 20
stt = 0.025/100
transaction_charge = 0.000325/100
stamp = 0.003/100
sebi_charges = 10

for index, row in data.iterrows():

    current_price = row['Close']
    pre_incrase = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    
    if pre_incrase >= target_prencantage and long_hold:
        long_hold = False
        profit_long = current_price-old_price_long
        print(f'selling at a price {current_price}. profit = {(current_price-old_price_long)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})

    
    if not long_hold:
        print('Got False going long')
        long_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_long = row['Close']
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'long','price':current_price,'order_value':int(order_value),'profit':0,'charges':charges,'profit_after_charges':-charges})


    pre_dec = ((-current_price+old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    if pre_dec >= target_prencantage and short_hold:
        short_hold = False
        profit_short = -current_price+old_price_short
        print(f'selling at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stamp_rs = order_value*stamp
        charges = brokerage+stamp_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_short*qty)-charges})
    

    
    if not short_hold:
        print('Got False going short')
        short_hold = True
        print(f'Buying at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = row['Close']
        order_value = qty*current_price
        tc = order_value*transaction_charge
        stt_rs = order_value*stt
        charges = brokerage+stt_rs+brokerage+sebi_charges
        trade_book.append({'date':index,'type':'short','price':current_price,'order_value':int(order_value),'profit':0,'charges':charges,'profit_after_charges':0-charges})

    
    

    if index.hour==15 and index.minute==29:
        if long_hold:
            long_hold = False
            profit_long = current_price-old_price_long
            print(f'Ending Trade for day {current_price}. profit = {(current_price-old_price_long)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stt_rs = order_value*stt
            charges = brokerage+stt_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':charges,'profit_after_charges':(profit_long*qty)-charges})


        if short_hold:
            short_hold = False
            profit_short = -current_price+old_price_short
            print(f'Ending Short at a price {current_price}. profit = {(-current_price+old_price_short)*qty} \n')
            order_value = qty*current_price
            tc = order_value*transaction_charge
            stamp_rs = order_value*stamp
            charges = brokerage+stamp_rs+brokerage+sebi_charges
            trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':charges,'profit_after_charges':(profit_short*qty)-charges})
            
    
        break




    
    


Got False going long
Buying at a price 46524.8515625. total buy price 1395745.546875 
 
Got False going short
Buying at a price 46524.8515625. total buy price 1395745.546875 
 
selling at a price 46580.55078125. profit = 1670.9765625 

Got False going long
Buying at a price 46580.55078125. total buy price 1397416.5234375 
 
selling at a price 46461.1015625. profit = 1912.5 

Got False going short
Buying at a price 46461.1015625. total buy price 1393833.046875 
 
selling at a price 46405.1484375. profit = 1678.59375 

Got False going short
Buying at a price 46405.1484375. total buy price 1392154.453125 
 
selling at a price 46350.1015625. profit = 1651.40625 

Got False going short
Buying at a price 46350.1015625. total buy price 1390503.046875 
 
selling at a price 46288.30078125. profit = 1854.0234375 

Got False going short
Buying at a price 46288.30078125. total buy price 1388649.0234375 
 
Ending Trade for day 46333.1015625. profit = -7423.4765625 

Ending Short at a price 46333.10

In [60]:
trade_book_df = pd.DataFrame(trade_book)

In [61]:
trade_book_df

,date,type,price,order_value,profit,charges,profit_after_charges
0,2024-03-19 09:15:00+05:30,long,46524.851562,1395745,0.000000,91.872366,-91.872366
1,2024-03-19 09:15:00+05:30,short,46524.851562,1395745,0.000000,398.936387,-398.936387
2,2024-03-19 09:44:00+05:30,exit long,46580.550781,1397416,1670.976562,399.354131,1271.622432
3,2024-03-19 09:44:00+05:30,long,46580.550781,1397416,0.000000,91.922496,-91.922496
4,2024-03-19 09:58:00+05:30,exit short,46461.101562,1393833,1912.500000,91.814991,1820.685009
5,2024-03-19 09:58:00+05:30,short,46461.101562,1393833,0.000000,398.458262,-398.458262
6,2024-03-19 10:07:00+05:30,exit short,46405.148438,1392154,1678.593750,91.764634,1586.829116
7,2024-03-19 10:07:00+05:30,short,46405.148438,1392154,0.000000,398.038613,-398.038613
8,2024-03-19 10:22:00+05:30,exit short,46350.101562,1390503,1651.406250,91.715091,1559.691159
9,2024-03-19 10:22:00+05:30,short,46350.101562,1390503,0.000000,397.625762,-397.625762


In [62]:
sum(trade_book_df['profit'])

0.0

In [63]:
sum(trade_book_df['profit_after_charges'])

-3429.5225125